In [3]:
%pip install opencv-python

  Using cached opencv_python-4.8.0.76-cp37-abi3-win_amd64.whl (38.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2 as cv
import mediapipe as mp


TypeError: 'numpy._DTypeMeta' object is not subscriptable

In [ ]:
import numpy as np
def image_overlay_third(img1,img2,location):
    h,w = img1.shape[:2]
    h1,w1 = img2.shape[:2]
    x,y = location
    roi = img1[y:y+h1 , x:x+w1]
    
    gray = cv.cvtColor(img2,cv.COLOR_BGR2GRAY)
    _,mask = cv.threshold(gray,150,255,cv.THRESH_BINARY)
    mask_inv = cv.bitwise_not(mask)
    
    img_bg = cv.bitwise_and(roi,roi,mask=mask)
    img_fg = cv.bitwise_and(img2,img2,mask=mask_inv)
    
    final = cv.add(img_bg,img_fg)
    
    img1[y:y+h1,x:x+w1] = final

    return img1

In [ ]:
if __name__ == '__main__':

    
    cap = cv.VideoCapture(0)
    
    overlay = cv.imread("./hairstyle.png")
    overlay = cv.resize(overlay ,(155,165))

    mpface = mp.solutions.face_detection
    face = mpface.FaceDetection()

    mpdraw = mp.solutions.drawing_utils
    
    while cap.isOpened():
        try:
            
            ret,img = cap.read()
            img = cv.resize(img ,(500,400))

            img3 = cv.cvtColor(img,cv.COLOR_BGR2RGB)
            results = face.process(img3)
            #print(results.detections)





            frame = cv.cvtColor(img,cv.COLOR_BGR2HSV_FULL)

            if results.detections:
                for id,detection in enumerate(results.detections):
                #mpdraw.draw_detection(frame,detection)
                    bbox = detection.location_data.relative_bounding_box
                    ih,iw,ic = img3.shape
                    cv.rectangle(img,(int(bbox.xmin *iw)-10,int(bbox.ymin *ih)-40),(int((bbox.xmin + bbox.width)* iw),int((bbox.ymin + bbox.height)*ih)),(255,0,255),2)
                    roi = img[int(bbox.ymin *ih)-40:int((bbox.ymin + bbox.height)*ih),int(bbox.xmin *iw)-10:int((bbox.xmin + bbox.width)* iw)]
                    after_overlay=image_overlay_third(img,overlay,location = (int(bbox.xmin *iw)-10,int(bbox.ymin *ih)-80))



            cv.imshow("img",after_overlay)
        except Exception:
            pass
        
    
        if cv.waitKey(1) == 27 :
            break
    cap.release()
    cv.destroyAllWindows()
